### Инструкция по подключению PostgreSQL и Apache Superset ###

1. Создаем сервер в облаке и подключаемся к нему через терминал
2. Устанавливаем Docker в соответствии со Step 1 из [инструкции](https://www.digitalocean.com/community/tutorials/how-to-install-and-use-docker-on-ubuntu-22-04)
3. Создаем хранилище (volume) для контейнера с БД: 
    ```
    docker volume create postgres_1_vol
    ```

4. Запускаем контейнер с PostgresDB:
    ```
    docker run -d \
        --name postgres_1 \
        -e POSTGRES_USER=postgres \
        -e POSTGRES_PASSWORD='123' \
        -e POSTGRES_DB=test_app \
        -v postgres_1_vol:/var/lib/postgresql \
        postgres:15
    ```
    `POSTGRES_USER`, `POSTGRES_PASSWORD` нужно запомнить – они понадобятся для подключения. 

5. Запускаем контейнер Superset:
    ```
    docker run --rm -d \
        -p 8080:8088 \
        -e "SUPERSET_SECRET_KEY=$(openssl rand -base64 42)" \
        --name superset \
        apache/superset
    ```

6. Последовательно запускаем команды для настройки Superset: создание пользователя, обновление внутренней базы, инициализация сервиса: 

    ```
    docker exec -it superset superset fab create-admin \
                --username admin \
                --firstname Superset \
                --lastname Admin \
                --email admin@superset.com \
                --password admin
    ```
    `username`, `password` понадобятся для авторизации в Superset. 

    Обновляем внутреннюю БД Superset:
    ```
    docker exec -it superset superset db upgrade
    ```

    Запускаем сервер Superset:
    ```
    docker exec -it superset superset init
    ```

7. Подключаем контейнеры в созданную сеть: 

    ```
    docker network connect <имя вашей сети> <имя контейнера postgres>
    ```

    ```
    docker network connect <имя вашей сети> <имя контейнера superset>
    ```

8. Добавление БД в контейнер с Postgres. Есть два способа:
    
   
    8.1 Зайти в контейнер, скачать, распаковать и добавить в Postgres базу прямо там. 
    Заходим в контейнер и запускаем `bash`  внутри: 
    ```
    docker exec -it <your_postgres_container_name> bash 
    ```
    Переходим в нужную папку (как на прошлой неделе): 
    ```
    cd /var/lib/postgresql/ 
    ```
    Обновляем и устанавливаем пакеты:
    ```
    apt-get update
    apt-get install -y wget  
    ```
    Скачиваем базу: 
    ```
    wget https://9c579ca6-fee2-41d7-9396-601da1103a3b.selstorage.ru/credit_clients.csv
    
    ```

    Переключаемся на пользователя `postgres` и создаем таблицу для базы: 
    ```
    su postgres

    CREATE TABLE customer_data (
    Date DATE,
    CustomerId INTEGER,
    Surname TEXT,
    CreditScore INTEGER,
    Geography TEXT,
    Gender TEXT,
    Age INTEGER,
    Tenure INTEGER,
    Balance FLOAT,
    NumOfProducts INTEGER,
    HasCrCard INTEGER,
    IsActiveMember INTEGER,
    EstimatedSalary FLOAT,
    Exited INTEGER);

    добавляем данные в таблицу

    \copy customer_data FROM 'credit_clients.csv' DELIMITER ',' CSV HEADER;

    ```


9. В локальном терминале прокидываем соединение: 
    ```
    ssh -L  8080:localhost:8080 <user_name>@<ip-address>
    ```

    В браузере переходим по адресу: 
    [`http://localhost:8080/login/`](http://localhost:8080/login/)

10. После входа на сайте Superset в правом вернем углу нажимаем на "+" -> DataBase+ и подключаем нашу базу данных

11. После подключения создаем датасет посредством SQL Lab запроса и выводим нужные нам таблицы